In [12]:
import tensorflow as tf
import math
import itertools
import numpy as np
from data_helpers import RWBatchGenerator

data_dir = 'data/stackexchange/datascience/'

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 3       # How many words to consider left and right.
num_skips = 4  # How many times to reuse an input to generate a label.

walks = RWBatchGenerator.read_walks("{}/random_walks.txt".format(data_dir))

vocabulary_size = len(set(itertools.chain(*walks)))

generator = RWBatchGenerator(walks, batch_size, num_skips, skip_window)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generator.next_batch()
        
        feed_dict = {train_inputs: batch_inputs,
                     train_labels: np.expand_dims(np.array(batch_labels), -1)}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = valid_examples[i]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = nearest[k]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  205.150756836
Nearest to 63: 2555, 613, 1839, 3474, 3987, 5698, 3475, 4370,
Nearest to 27: 351, 2498, 4440, 2578, 3, 3271, 1362, 5641,
Nearest to 22: 668, 4271, 1461, 4776, 1824, 1875, 1166, 512,
Nearest to 26: 1617, 5241, 5608, 2576, 5834, 4265, 4248, 1906,
Nearest to 98: 1451, 3781, 4037, 1589, 110, 2793, 5802, 5872,
Nearest to 53: 5247, 1330, 418, 1732, 5113, 4464, 2208, 4920,
Nearest to 33: 1773, 3414, 4662, 498, 3045, 6019, 4332, 5976,
Nearest to 50: 273, 809, 5267, 1278, 4969, 1337, 1301, 3785,
Nearest to 84: 59, 1010, 5633, 5150, 1347, 761, 1561, 5586,
Nearest to 6: 1460, 1341, 3026, 4743, 5776, 5624, 5177, 197,
Nearest to 52: 1873, 2495, 5842, 3757, 4565, 1462, 987, 1147,
Nearest to 7: 3134, 971, 510, 3843, 3103, 2141, 5169, 5171,
Nearest to 93: 762, 2805, 5887, 4151, 4198, 2103, 3720, 2804,
Nearest to 38: 2870, 1917, 4864, 1539, 3563, 155, 1858, 4388,
Nearest to 45: 4753, 5443, 3582, 4409, 1398, 3134, 5076, 1104,
Nearest to 9: 111, 4073, 

KeyboardInterrupt: 